In [ ]:
!pip install transformers
import json
import copy
import os
import math
import torch
import torch.nn as nn
from torchtext.legacy import data
import random
import math
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch import Tensor 
import pandas as pd
import torch.nn.functional as F
from transformers import (WEIGHTS_NAME, BertConfig, BertModel, BertPreTrainedModel, BertTokenizer)
from torch.nn import CrossEntropyLoss
from torch.utils.data import TensorDataset, RandomSampler, DataLoader, SequentialSampler

In [ ]:
import pandas as pd
#from torchtext.data.metrics import bleu_score
from sklearn.model_selection import train_test_split
#from torch.utils.tensorboard import SummaryWriter
from google.colab import drive
drive.mount('/content/gdrive/')

%cd /content/gdrive/My Drive/Kaggle


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/My Drive/Kaggle


In [ ]:
ADDITIONAL_SPECIAL_TOKENS = ['<e1>', "</e1>", "<e2>", "</e2>"]
def load_tokenizer():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenizer.add_special_tokens({"additional_special_tokens": ADDITIONAL_SPECIAL_TOKENS})
    return tokenizer

In [ ]:
class InputFeatures(object):
    """
    A single set of features of data.
    Args:
        input_ids: Indices of input sequence tokens in the vocabulary.
        attention_mask: Mask to avoid performing attention on padding token indices.
            Mask values selected in ``[0, 1]``:
            Usually  ``1`` for tokens that are NOT MASKED, ``0`` for MASKED (padded) tokens.
        token_type_ids: Segment token indices to indicate first and second portions of the inputs.
    """

    def __init__(self, input_ids, attention_mask, label_id, e1_mask, e2_mask):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.label_id = label_id
        self.e1_mask = e1_mask
        self.e2_mask = e2_mask

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"

In [ ]:
def convert_inputs_to_features(
    texts,
    label_id,
    max_seq_len,
    tokenizer,
    cls_token="[CLS]",
    sep_token="[SEP]",
    pad_token=0,
    mask_padding_with_zero=True,
):
    features = []
    #print(texts)
    for (ex_index, example) in enumerate(texts):

        #print(ex_index, example)
        tokens_a = tokenizer.tokenize(example)
        #print(tokens_a)

        e11_p = tokens_a.index("<e1>")  # the start position of entity1
        e12_p = tokens_a.index("</e1>")  # the end position of entity1
        e21_p = tokens_a.index("<e2>")  # the start position of entity2
        e22_p = tokens_a.index("</e2>")  # the end position of entity2

        # Replace the token
        tokens_a[e11_p] = "$"
        tokens_a[e12_p] = "$"
        tokens_a[e21_p] = "#"
        tokens_a[e22_p] = "#"

        # Add 1 because of the [CLS] token
        e11_p += 1
        e12_p += 1
        e21_p += 1
        e22_p += 1

        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = 2
        if len(tokens_a) > max_seq_len - special_tokens_count:
            tokens_a = tokens_a[: (max_seq_len - special_tokens_count)]

        tokens = [cls_token] + tokens_a + [sep_token]

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real tokens are attended to.
        attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_len - len(input_ids)
        input_ids = input_ids + ([pad_token] * padding_length)
        attention_mask = attention_mask + ([0 if mask_padding_with_zero else 1] * padding_length)

        # e1 mask, e2 mask
        e1_mask = [0] * len(attention_mask)
        e2_mask = [0] * len(attention_mask)

        for i in range(e11_p, e12_p + 1):
            e1_mask[i] = 1
        for i in range(e21_p, e22_p + 1):
            e2_mask[i] = 1

        assert len(input_ids) == max_seq_len, "Error with input length {} vs {}".format(len(input_ids), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(
            len(attention_mask), max_seq_len
        )

        features.append(
            InputFeatures(
                input_ids=input_ids,
                attention_mask=attention_mask,
                label_id=label_id,
                e1_mask=e1_mask,
                e2_mask=e2_mask,
            )
        )

    return features


In [ ]:
'''
#!pip install torch==1.3.1+cpu -f https://download.pytorch.org/whl/torch_stable.html
#!pip install inltk

from inltk.inltk import identify_language, tokenize, setup_language, reset_language_identifying_models
reset_language_identifying_models()
#setup_language('hi')
#setup_language('bn')
#setup_language('te')
#identify_language('hello, its english')
'''

#df = pd.read_csv('train.tsv', sep='\t')
df_en = pd.read_csv('en.tsv', sep='\t')
#df = pd.concat([df, df_en])

#inputs = []
'''
lang = identify_language(df['Sentence'].iloc[1]).run_until_complete()

for i in range(len(df)):
  lang = identify_language(df['Sentence'].iloc[i])
  sentence = ' '.join(tokenize(df['Sentence'].iloc[i], lang))
  inputs.append(df['NER1'].iloc[i]+'-'+df['NER2'].iloc[i]+' '+sentence)
'''
'''
for i in range(len(df)):
  inputs.append(df['NER1'].iloc[i]+'-'+df['NER2'].iloc[i]+' '+df['Sentence'].iloc[i])
df['CLS_sentence'] = inputs

df = df[['CLS_sentence', 'Relation']]
'''
labels = df_en['Relation'].unique().tolist()
labels_map = {}
for i in range(len(labels)):
  labels_map[labels[i]] = i

ids = []

for i in range(len(df_en)):
  ids.append(labels_map[df_en['Relation'].iloc[i]])
df_en['Label_id'] = ids

df = df_en[['Sentence', 'Label_id']]

df = df.sample(frac=1).reset_index(drop=True)

#print(df['CLS_sentence'].iloc[0])
              
#train, test = train_test_split(df, test_size = 0.2, shuffle = True, random_state = 42)

print()

In [ ]:
def Load_Dataset():
    tokenizer = load_tokenizer()
    features = convert_inputs_to_features(df['Sentence'].tolist(), df['Label_id'].tolist(), 120, tokenizer)
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_e1_mask = torch.tensor([f.e1_mask for f in features], dtype=torch.long)  # add e1 mask
    all_e2_mask = torch.tensor([f.e2_mask for f in features], dtype=torch.long)  # add e2 mask

    all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)

    dataset = TensorDataset(
        all_input_ids,
        all_attention_mask,
        all_label_ids,
        all_e1_mask,
        all_e2_mask
    )
    return dataset

In [ ]:
train_dataset = Load_Dataset()

In [ ]:
PAD_TOKEN, PAD_INDEX = '[PAD]', 0
UNK_TOKEN, UNK_INDEX = '[UNK]', 1
MASK_TOKEN, MASK_INDEX = '[MASK]', 2
CLS_TOKEN, CLS_INDEX = '[CLS]', 3
SEP_TOKEN, SEP_INDEX = '[SEP]', 4

In [ ]:
class PositionalEmbedding(nn.Module):

    def __init__(self, max_len, hidden_size, ):
        super(PositionalEmbedding, self).__init__()
        self.positional_embedding = nn.Embedding(max_len, hidden_size)
        positions = torch.arange(0, max_len)
        self.register_buffer('positions', positions)

    def forward(self, sequence):
        batch_size, seq_len = sequence.size()
        positions = self.positions[:seq_len].unsqueeze(0).repeat(batch_size, 1)
        return self.positional_embedding(positions)

In [ ]:
class GELU(nn.Module):

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))

class TransformerEncoder(nn.Module):

    def __init__(self, layers_count, d_model, heads_count, d_ff, dropout_prob):
        super(TransformerEncoder, self).__init__()

        self.d_model = d_model
        self.encoder_layers = nn.ModuleList(
            [TransformerEncoderLayer(d_model, heads_count, d_ff, dropout_prob) for _ in range(layers_count)]
        )

    def forward(self, sources, mask):
        """Transformer bidirectional encoder
        args:
           sources: embedded_sequence, (batch_size, seq_len, embed_size)
        """
        for encoder_layer in self.encoder_layers:
            sources = encoder_layer(sources, mask)

        return sources


class TransformerEncoderLayer(nn.Module):

    def __init__(self, d_model, heads_count, d_ff, dropout_prob):
        super(TransformerEncoderLayer, self).__init__()

        self.self_attention_layer = Sublayer(MultiHeadAttention(heads_count, d_model, dropout_prob), d_model)
        self.pointwise_feedforward_layer = Sublayer(PointwiseFeedForwardNetwork(d_ff, d_model, dropout_prob), d_model)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, sources, sources_mask):
        # x: (batch_size, seq_len, d_model)

        sources = self.self_attention_layer(sources, sources, sources, sources_mask)
        sources = self.dropout(sources)
        sources = self.pointwise_feedforward_layer(sources)

        return sources


class Sublayer(nn.Module):

    def __init__(self, sublayer, d_model):
        super(Sublayer, self).__init__()

        self.sublayer = sublayer
        self.layer_normalization = LayerNormalization(d_model)

    def forward(self, *args):
        x = args[0]
        x = self.sublayer(*args) + x
        return self.layer_normalization(x)


class LayerNormalization(nn.Module):

    def __init__(self, features_count, epsilon=1e-6):
        super(LayerNormalization, self).__init__()

        self.gain = nn.Parameter(torch.ones(features_count))
        self.bias = nn.Parameter(torch.zeros(features_count))
        self.epsilon = epsilon

    def forward(self, x):

        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)

        return self.gain * (x - mean) / (std + self.epsilon) + self.bias


class MultiHeadAttention(nn.Module):

    def __init__(self, heads_count, d_model, dropout_prob, mode='self-attention'):
        super(MultiHeadAttention, self).__init__()

        assert d_model % heads_count == 0
        assert mode in ('self-attention', 'memory-attention')

        self.d_head = d_model // heads_count
        self.heads_count = heads_count
        self.mode = mode
        self.query_projection = nn.Linear(d_model, heads_count * self.d_head)
        self.key_projection = nn.Linear(d_model, heads_count * self.d_head)
        self.value_projection = nn.Linear(d_model, heads_count * self.d_head)
        self.final_projection = nn.Linear(d_model, heads_count * self.d_head)
        self.dropout = nn.Dropout(dropout_prob)
        self.softmax = nn.Softmax(dim=3)

        self.attention = None
        # For cache
        self.key_projected = None
        self.value_projected = None

    def forward(self, query, key, value, mask=None, layer_cache=None):
        """
        Args:
            query: (batch_size, query_len, model_dim)
            key: (batch_size, key_len, model_dim)
            value: (batch_size, value_len, model_dim)
            mask: (batch_size, query_len, key_len)
        """
        # print('attention mask', mask)
        batch_size, query_len, d_model = query.size()

        d_head = d_model // self.heads_count

        query_projected = self.query_projection(query)
        # print('query_projected', query_projected.shape)
        if layer_cache is None or layer_cache[self.mode] is None:  # Don't use cache
            key_projected = self.key_projection(key)
            value_projected = self.value_projection(value)
        else:  # Use cache
            if self.mode == 'self-attention':
                key_projected = self.key_projection(key)
                value_projected = self.value_projection(value)

                key_projected = torch.cat([key_projected, layer_cache[self.mode]['key_projected']], dim=1)
                value_projected = torch.cat([value_projected, layer_cache[self.mode]['value_projected']], dim=1)
            elif self.mode == 'memory-attention':
                key_projected = layer_cache[self.mode]['key_projected']
                value_projected = layer_cache[self.mode]['value_projected']

        # For cache
        self.key_projected = key_projected
        self.value_projected = value_projected

        batch_size, key_len, d_model = key_projected.size()
        batch_size, value_len, d_model = value_projected.size()

        query_heads = query_projected.view(batch_size, query_len, self.heads_count, d_head).transpose(1, 2)  # (batch_size, heads_count, query_len, d_head)
        key_heads = key_projected.view(batch_size, key_len, self.heads_count, d_head).transpose(1, 2)  # (batch_size, heads_count, key_len, d_head)
        value_heads = value_projected.view(batch_size, value_len, self.heads_count, d_head).transpose(1, 2)  # (batch_size, heads_count, value_len, d_head)

        attention_weights = self.scaled_dot_product(query_heads, key_heads)  # (batch_size, heads_count, query_len, key_len)

        if mask is not None:
            mask_expanded = mask.unsqueeze(1).expand_as(attention_weights)
            attention_weights = attention_weights.masked_fill(mask_expanded, -1e18)

        self.attention = self.softmax(attention_weights)  # Save attention to the object
        attention_dropped = self.dropout(self.attention)
        context_heads = torch.matmul(attention_dropped, value_heads)  # (batch_size, heads_count, query_len, d_head)
        context_sequence = context_heads.transpose(1, 2).contiguous()  # (batch_size, query_len, heads_count, d_head)
        context = context_sequence.view(batch_size, query_len, d_model)  # (batch_size, query_len, d_model)
        final_output = self.final_projection(context)

        return final_output

    def scaled_dot_product(self, query_heads, key_heads):
        """
        Args:
             query_heads: (batch_size, heads_count, query_len, d_head)
             key_heads: (batch_size, heads_count, key_len, d_head)
        """
        key_heads_transposed = key_heads.transpose(2, 3)
        dot_product = torch.matmul(query_heads, key_heads_transposed)  # (batch_size, heads_count, query_len, key_len)
        attention_weights = dot_product / np.sqrt(self.d_head)
        return attention_weights


class PointwiseFeedForwardNetwork(nn.Module):

    def __init__(self, d_ff, d_model, dropout_prob):
        super(PointwiseFeedForwardNetwork, self).__init__()

        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.Dropout(dropout_prob),
            GELU(),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout_prob),
        )

    def forward(self, x):
        """
        Args:
             x: (batch_size, seq_len, d_model)
        """
        return self.feed_forward(x)

In [ ]:
def pad_masking(x):
    # x: (batch_size, seq_len)
    padded_positions = x == PAD_INDEX
    return padded_positions.unsqueeze(1)

In [ ]:
class BERT(nn.Module):

    def __init__(self, encoder, token_embedding, positional_embedding, hidden_size, vocabulary_size, num_lables):
        super(BERT, self).__init__()

        self.encoder = encoder
        self.token_embedding = token_embedding
        self.positional_embedding = positional_embedding
        self.token_prediction_layer = nn.Linear(hidden_size, vocabulary_size)
        self.classification_layer = nn.Linear(hidden_size, num_labels)

    def forward(self, inputs):
        sequence, segment = inputs
        token_embedded = self.token_embedding(sequence)
        positional_embedded = self.positional_embedding(sequence)
        embedded_sources = token_embedded + positional_embedded

        mask = pad_masking(sequence)
        encoded_sources = self.encoder(embedded_sources, mask)
        token_predictions = self.token_prediction_layer(encoded_sources)
        classification_embedding = encoded_sources[:, 0, :]
        classification_output = self.classification_layer(classification_embedding)
        return token_predictions, classification_output

In [ ]:
def build_model(layers_count, hidden_size, heads_count, d_ff, dropout_prob, max_len, vocabulary_size):
    token_embedding = nn.Embedding(num_embeddings=vocabulary_size, embedding_dim=hidden_size)
    positional_embedding = PositionalEmbedding(max_len=max_len, hidden_size=hidden_size)
    segment_embedding = SegmentEmbedding(hidden_size=hidden_size)

    encoder = TransformerEncoder(
        layers_count=layers_count,
        d_model=hidden_size,
        heads_count=heads_count,
        d_ff=d_ff,
        dropout_prob=dropout_prob)

    bert = BERT(
        encoder=encoder,
        token_embedding=token_embedding,
        positional_embedding=positional_embedding,
        segment_embedding=segment_embedding,
        hidden_size=hidden_size,
        vocabulary_size=vocabulary_size)

    return bert

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# device="cpu"

In [ ]:
#%%writefile app.py
def save_checkpoint(state, filename="my_check.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(PATH, model, optimizer):
    print("=> Loading checkpoint")
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    epoch = checkpoint['epoch']
    return epoch, model, optimizer

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [ ]:
adam_epsilon=1e-8
warmup_steps=0
max_steps = -1
learning_rate = 3e-5

train_sampler = RandomSampler(train_dataset)

train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size= train_batch_size)

if max_steps > 0:
    t_total = max_steps
    num_train_epochs = max_steps // len(train_dataloader) + 1
else:
    t_total = len(train_dataloader) // num_train_epochs
        
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters()
                    if not any(nd in n for nd in no_decay)], 'weight_decay': 1e-3},
        {'params': [p for n, p in model.named_parameters()
                    if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
optimizer = AdamW(optimizer_grouped_parameters,
                      lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)


NameError: ignored

In [ ]:
def train(train_dataloader, optimizer, scheduler, train_dataset, model, tokenizer, num_train_epochs=5, max_grad_norm = 1.0):
    """ Train the model """
    train_dataloader = train_dataloader
    optimizer = optimizer
    scheduler = scheduler

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(int(num_train_epochs), desc="Epoch")

    # Added here for reproductibility (even between python 2 and 3)
    set_seed(42)
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration")
        for step, batch in enumerate(epoch_iterator):
            model.train()
            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      # XLM and RoBERTa don't use segment_ids
                      'token_type_ids': batch[2],
                      'labels':      batch[3],
                      'e1_mask': batch[4],
                      'e2_mask': batch[5],
                      }

            outputs = model(**inputs)
            # model outputs are always tuple in pytorch-transformers (see doc)
            loss = outputs[0]

            loss.backward()
            torch.nn.utils.clip_grad_norm_(
                model.parameters(), max_grad_norm)

            tr_loss += loss.item()

            optimizer.step()
            scheduler.step()  # Update learning rate schedule
            model.zero_grad()
            global_step += 1

            if max_steps > 0 and global_step > max_steps:
                epoch_iterator.close()
                break
        if max_steps > 0 and global_step > max_steps:
            train_iterator.close()
            break
    return global_step, tr_loss / global_step

In [ ]:
bertconfig = BertConfig.from_pretrained("bert-base-uncased", num_labels=len(labels))

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", config=bertconfig)

train(train_dataloader, train_dataloader, optimizer, scheduler, train_dataset, model, tokenizer, num_train_epochs=5, max_grad_norm = 1.0)

ValueError: ignored

In [ ]:
# %%writefile app.py
path = %pwd
PATH = os.path.join(path, 'my_check.pth.tar')
epoch, model, opt = load_checkpoint(PATH, model, opt)
df = pd.read_csv('valid.tsv', sep='\t')

outputs = []
for i in range(len(df)):
  outputs.append(predict(model, get_inp_sen_tensor(df['Sentence'].iloc[i]))[:-5])

df['Relation'] = outputs
df['Relation'].to_csv('submission.csv')